[View in Colaboratory](https://colab.research.google.com/github/dattarajrao/keras2kubernetes/blob/master/TensorFlow_CPU_vs_GPU_times.ipynb)

# ***Import TensorFlow library and check if Cloud runtime has a GPU***

In [7]:
import tensorflow as tf
import numpy as np

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
else:
  print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# ***Load the CIFAR-10 Dataset in memory***

In [0]:
cifar = tf.keras.datasets.cifar10

(x_train, y_train),(x_test, y_test) = cifar.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# ***Build the Convolutional Neural Network Model***

In [23]:
model = tf.keras.models.Sequential([
    
  tf.keras.layers.Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.Dropout(0.25),

  tf.keras.layers.Conv2D(64, (3, 3), padding='same'),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(64, (3, 3)),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.Dropout(0.25),
    
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_9 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
activation_10 (Activation)   (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 14, 64)        36928     
__________

# ***Train the Model and record Training Time***

In [10]:
import datetime

# start training
st_time = datetime.datetime.now()

model.fit(x_train, y_train, epochs=1)

# record time after training
end_time = datetime.datetime.now()

print('Training time = %s'%(end_time-st_time))

model.evaluate(x_test, y_test)

Epoch 1/1
50000/50000 [==============================] - 24s 478us/step - loss: 1.4407 - acc: 0.4763
Training time = 0:00:24.654200
10000/10000 [==============================] - 1s 141us/step


[1.151843452835083, 0.582]

# ***Save the Model file***

In [0]:
model.save('my_cifar_cnn.h5')

# ***Load Model and do Inference - Record times***

In [22]:
from tensorflow.keras.models import load_model

newmodel = load_model('my_cifar_cnn.h5')

# start inference - for 1000 tests
st_time = datetime.datetime.now()

for count in range(1000):
  prediction = newmodel.predict(np.expand_dims(x_test[count], axis=0))
  #print(y_test[count][0])
  #print(np.argmax(prediction))
  #print('----')
  
# record time after inference
end_time = datetime.datetime.now()

print('Inference time = %s'%(end_time-st_time))

Inference time = 0:00:02.178994
